In [1]:
import sys
import os
import glob
import json

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, os.path.join(project_root, 'src'))

import chunking.chunker as chunk
import embedding.embedder as emb
import loaders.loaders as load
import loaders.proccesing as proc
#import retrievers.retrievers as ret
import vector_store_client.vector_client as vsc
# import pdf_processing.pdf_to_json as pdf
# import text_preprocessing.text_preprocessing as tp

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\gigle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
c:\Users\gigle\anaconda3\envs\mia\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
input_path = "../data/reglamentacion/"
output_path = "../data/staging/"

In [3]:
# processor = proc.Processing(ruta=input_path, output=output_path)

# # Convert PDFs to JSON
# processor.pdf_to_json()

# # Clean the JSON data
# processor.clean_data()

In [4]:
all_chunks = []

for file in glob.glob(output_path+'*.json'):
    # Load a cleaned JSON file
    print(file)
    with open(file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Pass it to Chunker
    chunker = chunk.Chunker(data, max_chunk_size=600)
    chunks = chunker.chunking_hybrid()

    all_chunks.extend(chunks)

../data/staging\capitulo7.json
../data/staging\capitulo1.json
../data/staging\capitulo10.json
../data/staging\capitulo11.json
../data/staging\capitulo12.json
../data/staging\capitulo13.json
../data/staging\capitulo14.json
../data/staging\capitulo15.json
../data/staging\capitulo16.json
../data/staging\capitulo17.json
../data/staging\capitulo18.json
../data/staging\capitulo19.json
../data/staging\capitulo2.json
../data/staging\capitulo20.json
../data/staging\capitulo21.json
../data/staging\capitulo22.json
../data/staging\capitulo3.json
../data/staging\capitulo4.json
../data/staging\capitulo5.json
../data/staging\capitulo6.json
../data/staging\capitulo8.json
../data/staging\capitulo9.json


In [5]:
print(len(all_chunks))
all_chunks[0]

33212


{'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
 'contenido': 'consideran Aceites alimenticios o Aceites comestibles, los admitidos como aptos para'}

In [6]:
embedder = emb.Embedder()
enriched_chunks = embedder.generate_embeddings(chunks)

c:\Users\gigle\anaconda3\envs\mia\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
len(enriched_chunks)

1473

In [8]:
all_chunks

[{'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
  'contenido': 'consideran Aceites alimenticios o Aceites comestibles, los admitidos como aptos para'},
 {'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
  'contenido': 'por el presente y los que en el futuro sean aceptados como tales por la'},
 {'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
  'contenido': 'aceites alimenticios se obtendrán a partir de semillas o frutos oleaginosos'},
 {'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
  'contenido': 'de elaboración que se ajusten a las condiciones de higiene establecidas por'},
 {'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
  'contenido': 'Presentarán aspecto límpido a 25C, sabor y olor agr adables y contendrán solamente'},
 {'metadata': {'type': 'narrative', 'filename': 'capitulo7.pdf'},
  'contenido': 'propios del aceite que integra la composición de las semillas o frutos de'},
 {'metadata': {'type': '

In [9]:
vector_client = vsc.VectorStoreClient(type="qdrant")

# Insert embeddings into the vector store
vector_client.insert_embeddings(enriched_chunks)

https://60f9378e-8ae8-461c-a69b-d231d6d3617d.europe-west3-0.gcp.cloud.qdrant.io


True

In [10]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333, prefer_grpc=True, https=False)

In [11]:

# Perform a search
query = "Que dice la ley respecto a los aceites?"
results = vector_client.search(query)

# Display search results
for result in results:
    print(result)

{'id': '5edafe9a-59dc-49ee-979e-756698aafda2', 'score': 0.6645614504814148, 'content': 'a la espinaca o con espinaca, con acelga o a la'}
{'id': '14dcab55-2158-47f4-9811-76fbc87fa968', 'score': 0.6378410458564758, 'content': 'para el consumo, que habrá ap robado, en cada caso, la autoridad sanitaria'}
{'id': '5270cf8f-42c6-4ace-be0b-004029d70f36', 'score': 0.6310109496116638, 'content': '1. Cuando contengan edulcorantes, deberán llevar la leyenda: Galletitas dulces o'}
{'id': '91e4c9c6-dca5-46a1-b259-1cfb3090838d', 'score': 0.6304302215576172, 'content': 'con o sin el agregado de levadura, con o'}
{'id': '031ee38d-ae7a-4997-8c32-887eab85c529', 'score': 0.627261757850647, 'content': 'en el rótulo deberá consignarse: mes y año de'}
